In [147]:
# 나의 구글 드라이브를 구글 본사 서버와 연결하는 작업
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
# 현재 실행 경로 확인하기
import os
print(os.getcwd())

/content/drive/My Drive/nlp


In [149]:
# 현재 작업 폴더 nlp로 경로 이동하기
%cd '/content/drive/My Drive/nlp'

/content/drive/My Drive/nlp


In [150]:
print(os.getcwd())

/content/drive/My Drive/nlp


In [151]:
# 필요한 라이브러리 임폴트
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [152]:
"""
KoNLpy(코엔엘파이) 라이브러리 설치
KoNLpy(코엔엘파이) 라이브러리 설치 : 한국어 형태소 분석기로서, 한글 자연어 처리를 위해
만들어진 라이브러리
"""

# KoNLpy 라이브러리 설치
!pip install konlpy

In [153]:
# 필요한 라이브러리 임폴트
import konlpy
from konlpy.tag import Kkma, Okt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [154]:
# 한글 형태소 분석기 생성

kkma = Kkma()
okt = Okt()

In [208]:
# pandas.read_csv() 함수를 사용하여 데이터프레임 생성하기
df = pd.read_csv('3rd_project_ver001.csv')
#pd.options.display.max_columns = None
print("df.title = \n", df.title)
print("df.content = \n", df.content)

df.title = 
 0            (주)광주글로벌모터스 AI역량검사 지원부스 운영<7.26(월)~7.27(화)>
1          (주)광주글로벌모터스 채용설명 취업특강<7.23(금), 7.26(월)> 운영 안내
2       아름다운 호수의 여왕 술루우수우와 전쟁으로 폐허가 된 마을 아이들이 펼치는 우정 이야기
3                 무엇이든지 될 수 있는 ‘네네네’ 숲에 사는 세 친구의 재미난 놀이!
4      신명과 함께 깊이가 있고, 여운을 남기며, 여러 장르의 춤이 뒤섞여 아름다운 조화를...
                             ...                        
140                    [첨단] 첨단도서관 제3기 독서문화프로그램 수강생 모집 안내
141                [2021 국민참여 청렴콘텐츠 공모전] 2차 소문내기 이벤트 시작!
142            [2021 국민참여 청렴콘텐츠 공모전] 1차 소문내기 이벤트 당첨자 발표!
143                 [2021 국민참여 청렴콘텐츠 공모전] 수필, 시 분야 작성 양식
144                      [2021 국민참여 청렴콘텐츠 공모전] 소문내기 이벤트!
Name: title, Length: 145, dtype: object
df.content = 
 0      (주)광주글로벌모터스 AI 역량검사 지원 부스 운영】\n  ■ 대상 : ㈜광주글로벌...
1      〔채용설명 및 AI 역량검사 특강〕실시\n(줌(zoom)활용 비대면 강의)\n■ 특...
2      기간\n2021.7.23.(금) ~ 7.25.(일)\n시간\n금 19:30\n토,일...
3      기간\n2021.7.24.(토) - 7.25.(일)\n시간\n24(토) 11:00,...
4      기간\n2021. 7.28.(수)\n시간\n11:00 - 12:30\n장소\n예술극...
                    

# 타이틀 DB 

In [195]:
#데이터 (타이틀) 전처리 
import re
sentences = [data for data in df.title]

for sentence in sentences:
  for data in sentence:
    #print("data = ", data)
    re.sub('[^0-9a-zA-Zㄱ-힗]+', '', data)
    cleaned_text.append(data)


text = sentences

stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', data)

print("타이틀:",text)
print("타이틀갯수:",len(text))

타이틀: ['(주)광주글로벌모터스 AI역량검사 지원부스 운영<7.26(월)~7.27(화)>', '(주)광주글로벌모터스 채용설명 취업특강<7.23(금), 7.26(월)> 운영 안내', '아름다운 호수의 여왕 술루우수우와 전쟁으로 폐허가 된 마을 아이들이 펼치는 우정 이야기', '무엇이든지 될 수 있는 ‘네네네’ 숲에 사는 세 친구의 재미난 놀이!', '신명과 함께 깊이가 있고, 여운을 남기며, 여러 장르의 춤이 뒤섞여 아름다운 조화를 이뤄낸다.', '필리핀 전래동화 ‘바닷물은 왜 짤까?’를 바탕으로 인간과 자연의 평화로운 공존을 이야기하는 어린이음악극', '(의료기관)장애인학대 관련 취업제한 제도 안내', '백신접종 및 개인정보활용 동의서 서식', '2021년 광산구 고혈압·당뇨병 등록교육센터 인력 채용공고', 'Mom health 리더 임신부 건강증진프로그램 대상자 모집', '광산구 보건소 금연클리닉과 함께하실 대학생 자원봉사자를 구합니다.', '코로나19 예방접종 이상반응 피해보상 구비서류 안내', '어르신 폐렴구균(PPSV23) 국가예방접종 지원사업 위탁의료기관 안내', '2021 7월 임산부 건강교실', '(의료기관)아동학대, 긴급복지 신고의무자 교육 및 범죄 조회 공지', '[채용] 광주청소년삶디자인센터 벼리(직원)를 찾습니다', '[모집] 오감을 깨워, 먹다 – ‘심지’의 공유부엌 실험 프로젝트', '[상시모집] 친구와 요리하고 베이킹하고 싶다면? “요리하는 날”', '[상시모집] 청소년 동아리를 찾습니다', '[상시모집] 의류 수선이 필요하나요? ‘열린 수선방’으로 오세요😊', '[모집] 2021 일들의 사전 프로젝트 (3.3.~ 상시접수)', '[안내](MBC) 2021 하반기 국내제작 애니메이션 정기공모', '[안내] 광주역 도시재생 현장지원센터 교육 수요조사', '[안내] 투자진흥지구 발전위원회 위원 공모', '[안내] 2021년 여성가족부 「가족친화인증」신청(2차) 안내', '[안내] 「2021 소통 콘텐츠 공모전」주제 제안 안내', '[안내

# 내용 DB

In [215]:
#데이터 전처리 - 특수 문자 제거 하기 

import re
sentences2 = [data for data in df.content]

# for sentence in sentences2:
#   for data in sentence:
#     #print("data = ", data)
#     re.sub('[^0-9a-zA-Zㄱ-힗]+', '', data)
#     cleaned_text.append(data)


text2 = sentences2

stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', data)








import re

base_data = [data for data in df.content]
#print("base_data = \n", base_data[0])
print('\n')

# words1=base_data.replace(" ","").replace("  ","").replace("   ","").replace("    ","").replace("    ","").replace("    ","").replace("    ","").replace("     ","").replace("     ","")
re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', words1)

# print(base_data)
# sentences = []
# for data in base_data:    
#   sentence_str = str(data)
#   sentence = sentence_str.split('\n')
#   sentences.append(sentence)

print("내용 갯수:",len(base_data))
print("내용:",base_data)

# # 특수 문자가 제거된 문장 생성
# cleaned_text = []

# for sentence in sentences:
#   for data in sentence:
#     #print("data = ", data)
#     re.sub('[^0-9a-zA-Zㄱ-힗]+', '', data)
#     cleaned_text.append(data)

# print("내용 갯수:",len(cleaned_text))
# print("내용 :",cleaned_text)

text = sentences[1]

# stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', data)

# print(text)


# 결과 확인하기
# for data in cleaned_text:
#   print(data)

# str_0 = " ".join([string.strip() for string in sentences)

stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
words = ""

# for data in str_0:
#   if data not in stop_words:
#     words += data
#   else:
#     continue

words1=words.replace(" ","").replace("  ","").replace("   ","").replace("    ","").replace("    ","").replace("    ","").replace("    ","").replace("     ","").replace("     ","")





내용 갯수: 145
내용: ['(주)광주글로벌모터스 AI 역량검사 지원 부스 운영】\n  ■ 대상 : ㈜광주글로벌모터스 채용 지원자\n■ 장소 : 광주고용복지플러스센터 11층 상설채용관\n■ 접수 기간: 21.07.19.(월) ~ 21.7.23.(금), 선착순(접수기간 중에 조기 마감될 수 있음)\n■ 운영 기간: 21.07.26.(월) ~ 21.7.27.(화)\n■ 운영 시간: 09:30~11:30, 14:00~16:00 (검사소요시간 2시간, 코로나로 인원 제한 있음)\n■ 신청 방법 및 접수(광주고용복지플러스센터 취업지원총괄과)\n<워크넷(http://work.go.kr) 회원가입 및 구직신청 한 후 AI역량검사 참여 신청>\n- 이메일 접수 : ekbw0224@korea.kr(접수 기간 중 조기 마감 되는 경우 이메일 자동마감 반송)\n- 참여자 성명, 핸드폰 번호, 참여 희망 날짜, 시간 기재\n(날짜 및 시간은 고용센터에서 재조정하며, 본인 희망일(시간)과 달라질 수 있음)\n- 지원 부스 한정 운영으로 접수 기간 중에 조기 마감될 수 있으니 이점 양해 바랍니다.\n■ 개인 필수 준비물 : 본인 이어폰(마이크 기능 보유-4극 핀 방식 이어폰)\n■ 문의: 062-609-8550, 8765~8766\n※ 코로나 확산으로 가급적 AI역량검사를 개인공간(자택)을 선택하시고 장비 등의 사정으로 불가피한 경우에 한하여 고용센터 11층 상설채용관 부스를 이용하여 평가에 임해 주시기 바랍니다.(AI역량검사는 마스크를 착용하지 않고 진행되므로 개인 방역을 철저히 지켜 주시기 바랍니다.)', '〔채용설명 및 AI 역량검사 특강〕실시\n(줌(zoom)활용 비대면 강의)\n■ 특강대상: ㈜광주글로벌모터스 취업 준비 및 입사지원자\n■ 특강 일정(시간) (정원:회차별 40명)\n1회차: 21. 7월 23일(금) 오후 2시 ~ 4시\n2회차: 21. 7월 26일(월) 오전 10:00 ~ 12:00\n3회차: 21. 7월 26일(월) 오후 2시 ~ 4시\n■ 접수 마감일

#기간DB

In [158]:
#기간과 대상을 추출하기 위한 사전만들기
test_date = ['기간', '접수 기간','기간 :']

test_people = ['대상', '대상자', '대상 :', '대상자 :']

In [216]:
#기간 추출하기
sentences = [data for data in df.content]
print("sentences = \n", sentences)
print('\n')

stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
words = []
for sentence in sentences:  
  print("sentence = ",sentence)  
  sentence_str = str(sentence)
  temp = sentence_str.split('\n')
  print("개행 문자로 분리한 결과",temp)
  words.append(temp)

  #기간과 대상을 추출하기 위한 사전만들기
test_date = ['기간', '접수 기간','기간 :']

test_people = ['대상', '대상자', '대상 :', '대상자 :']

#멈추는 말
stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
words = []
for sentence in sentences:    
  words.append(str(sentence).split('\n'))
# print(words)
# 모듈 모음

period = []
test1 = []
test_per = []

# 7번째까지 글자 추출
for word in words:
  for data in word:
    test_per.append(data)
# print(slice_data)

# 비교 : 기간 - 딕셔너리하고 위치값 받기
for date in test_date:
  for versus in date:
    for i, name in enumerate(test_per):
      if versus in slice_data[i]:
        test1.append(versus+'@'+name+'@'+ test_per[i]+'@' + str(i))
print(test1[0])

# 검색 결과를 위치값으로 변환 why ? 우리가 필요한건 단어 사전에 있는 위치 값을 가지고 온 후에 sentences에서 해당 값만 추출하려고



sentences[0] = 
 ['(주)광주글로벌모터스 AI 역량검사 지원 부스 운영】', '  ■ 대상 : ㈜광주글로벌모터스 채용 지원자', '■ 장소 : 광주고용복지플러스센터 11층 상설채용관', '■ 접수 기간: 21.07.19.(월) ~ 21.7.23.(금), 선착순(접수기간 중에 조기 마감될 수 있음)', '■ 운영 기간: 21.07.26.(월) ~ 21.7.27.(화)', '■ 운영 시간: 09:30~11:30, 14:00~16:00 (검사소요시간 2시간, 코로나로 인원 제한 있음)', '■ 신청 방법 및 접수(광주고용복지플러스센터 취업지원총괄과)', '<워크넷(http://work.go.kr) 회원가입 및 구직신청 한 후 AI역량검사 참여 신청>', '- 이메일 접수 : ekbw0224@korea.kr(접수 기간 중 조기 마감 되는 경우 이메일 자동마감 반송)', '- 참여자 성명, 핸드폰 번호, 참여 희망 날짜, 시간 기재', '(날짜 및 시간은 고용센터에서 재조정하며, 본인 희망일(시간)과 달라질 수 있음)', '- 지원 부스 한정 운영으로 접수 기간 중에 조기 마감될 수 있으니 이점 양해 바랍니다.', '■ 개인 필수 준비물 : 본인 이어폰(마이크 기능 보유-4극 핀 방식 이어폰)', '■ 문의: 062-609-8550, 8765~8766', '※ 코로나 확산으로 가급적 AI역량검사를 개인공간(자택)을 선택하시고 장비 등의 사정으로 불가피한 경우에 한하여 고용센터 11층 상설채용관 부스를 이용하여 평가에 임해 주시기 바랍니다.(AI역량검사는 마스크를 착용하지 않고 진행되므로 개인 방역을 철저히 지켜 주시기 바랍니다.)']
data =  (주)광주글로벌모터스 AI 역량검사 지원 부스 운영】
data =    ■ 대상 : ㈜광주글로벌모터스 채용 지원자
data =  ■ 장소 : 광주고용복지플러스센터 11층 상설채용관
data =  ■ 접수 기간: 21.07.19.(월) ~ 21.7.23.(금), 선착순(접수기간 중에 조기 마감될 수 있음)
d

In [172]:
[0] * 3  + ['나는 사람이다']

[0, 0, 0, '나는 사람이다']

In [ ]:
#기간과 대상을 추출하기 위한 사전만들기
test_date = ['기간', '접수 기간','기간 :']

test_people = ['대상', '대상자', '대상 :', '대상자 :']

#멈추는 말
stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
words = []
for sentence in sentences:    
  words.append(str(sentence).split('\n'))
# print(words)
# 모듈 모음

period = []
test1 = []
test_per = []

# 7번째까지 글자 추출
for word in words:
  for data in word:
    test_per.append(data)
# print(slice_data)

# 비교 : 기간 - 딕셔너리하고 위치값 받기
for date in test_date:
  for versus in date:
    for i, name in enumerate(test_per):
      if versus in slice_data[i]:
        test1.append(versus+'@'+name+'@'+ test_per[i]+'@' + str(i))
print(test1[0])

# 검색 결과를 위치값으로 변환 why ? 우리가 필요한건 단어 사전에 있는 위치 값을 가지고 온 후에 sentences에서 해당 값만 추출하려고





    

기@■ 접수 기간: 21.07.19.(월) ~ 21.7.23.(금), 선착순(접수기간 중에 조기 마감될 수 있음)@■ 접수 기간: 21.07.19.(월) ~ 21.7.23.(금), 선착순(접수기간 중에 조기 마감될 수 있음)@3


In [ ]:
#기간 추출하기
sentences = [data for data in df.content]
#print("sentences = \n", sentences)
print('\n')

stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
words = []
for sentence in sentences:  
  #print("sentence = ",sentence)  
  sentence_str = str(sentence)
  temp = sentence_str.split('\n')
  #print("개행 문자로 분리한 결과",temp)
  words.append(temp)

#print(len(words))

### 첫번째 문장 가져오기
#print(words[0])

period = []

# 첫번째 문장의 첫번째 데이터 가져오기
#print(words[0][0])
# for data in words[0]:
#   if '기간' in data:
#     period.append(data)

for word in words:
  for data in word:
    if '접수 기간' in data[:7]:
      period.append(data)


print(period)
  # if "접수 기간" in sentence:
  #   words.append(sentence)  

#print(words)

# str_0 = " ".join([string.strip() for string in sentences[0]])


# stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']



# for data in str_0:
#   if data not in stop_words:
#     words += data
#   else:
#     continue



# words1=words.replace(" ","").replace("  ","").replace("   ","").replace("    ","").replace("    ","").replace("    ","").replace("    ","").replace("     ","").replace("     ","")


# print(words1)


# print(len(words1))
# words2=words1.split("기간")
# print(len(words2))






['■ 접수 기간: 21.07.19.(월) ~ 21.7.23.(금), 선착순(접수기간 중에 조기 마감될 수 있음)']


In [ ]:
#기간 추출하기
sentences = [data for data in df.content]
print("sentences = \n", sentences)
print('\n')

stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
words = []
for sentence in sentences:  
  print("sentence = ",sentence)  
  sentence_str = str(sentence)
  temp = sentence_str.split('\n')
  print("개행 문자로 분리한 결과",temp)
  words.append(temp)

sentences = 
 ['(주)광주글로벌모터스 AI 역량검사 지원 부스 운영】\n  ■ 대상 : ㈜광주글로벌모터스 채용 지원자\n■ 장소 : 광주고용복지플러스센터 11층 상설채용관\n■ 접수 기간: 21.07.19.(월) ~ 21.7.23.(금), 선착순(접수기간 중에 조기 마감될 수 있음)\n■ 운영 기간: 21.07.26.(월) ~ 21.7.27.(화)\n■ 운영 시간: 09:30~11:30, 14:00~16:00 (검사소요시간 2시간, 코로나로 인원 제한 있음)\n■ 신청 방법 및 접수(광주고용복지플러스센터 취업지원총괄과)\n<워크넷(http://work.go.kr) 회원가입 및 구직신청 한 후 AI역량검사 참여 신청>\n- 이메일 접수 : ekbw0224@korea.kr(접수 기간 중 조기 마감 되는 경우 이메일 자동마감 반송)\n- 참여자 성명, 핸드폰 번호, 참여 희망 날짜, 시간 기재\n(날짜 및 시간은 고용센터에서 재조정하며, 본인 희망일(시간)과 달라질 수 있음)\n- 지원 부스 한정 운영으로 접수 기간 중에 조기 마감될 수 있으니 이점 양해 바랍니다.\n■ 개인 필수 준비물 : 본인 이어폰(마이크 기능 보유-4극 핀 방식 이어폰)\n■ 문의: 062-609-8550, 8765~8766\n※ 코로나 확산으로 가급적 AI역량검사를 개인공간(자택)을 선택하시고 장비 등의 사정으로 불가피한 경우에 한하여 고용센터 11층 상설채용관 부스를 이용하여 평가에 임해 주시기 바랍니다.(AI역량검사는 마스크를 착용하지 않고 진행되므로 개인 방역을 철저히 지켜 주시기 바랍니다.)', '〔채용설명 및 AI 역량검사 특강〕실시\n(줌(zoom)활용 비대면 강의)\n■ 특강대상: ㈜광주글로벌모터스 취업 준비 및 입사지원자\n■ 특강 일정(시간) (정원:회차별 40명)\n1회차: 21. 7월 23일(금) 오후 2시 ~ 4시\n2회차: 21. 7월 26일(월) 오전 10:00 ~ 12:00\n3회차: 21. 7월 26일(월) 오후 2시 ~ 4시\n■ 접수 마감일: 개